In [1]:
import dotenv
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
dotenv.load_dotenv()

True

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-1.5-pro-lates")

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("insea-actuariat.pdf")
docs = loader.load()

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings


vectorstore = FAISS.from_documents(splits, GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [14]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is.
chat history: {chat_history}
input: {input}
answer:
"""
contextualize_q_prompt = ChatPromptTemplate.from_template(contextualize_q_system_prompt)


history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""



qa_prompt = ChatPromptTemplate.from_template(
    qa_system_prompt, placeholders=[MessagesPlaceholder("context")]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [16]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "quelles sont les principales caractéristiques de l'actuariat?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "tu peux le dire en anglais?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.5-pro-lates is not found for API version v1beta, or is not supported for GenerateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised NotFound: 404 models/gemini-1.5-pro-lates is not found for API version v1beta, or is not supported for GenerateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised NotFound: 404 models/gemini-1.5-pro-lates is not found for API version v1beta, or is not supported for GenerateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat

In [38]:
rag_chain.invoke("quelles sont les COMPETENCES des diplomés de la filiere actuaiat-finance de l'insea")

"Les diplômés de la filière actuariat-finance de l'INSEA acquièrent les compétences nécessaires pour occuper des postes de cadres supérieurs dans les secteurs bancaires et des assurances. Ils maîtrisent les aspects techniques et managériaux, et sont capables de développer des modèles dans divers contextes grâce à leurs compétences mathématiques, informatiques et statistiques. La formation leur permet également de dominer les aspects juridiques, comptables, fiscaux et commerciaux liés à leur domaine."